In [ ]:
import azureml.core
print(azureml.core.VERSION)

In [ ]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
ws.get_details()

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies
env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
myenv = env.clone("system_managed_scikit")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("azureml-core")
myenv.python.conda_dependencies=conda_dep
myenv.register(workspace=ws)

In [ ]:
%%writefile train.py

import joblib

from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
import sklearn

from azureml.core import Dataset
from azureml.core.run import Run
import numpy as np
import sklearn

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

dataset_x, dataset_y = load_diabetes(return_X_y=True)

model = Ridge().fit(dataset_x, dataset_y)

joblib.dump(model, 'sklearn_regression_model.pkl')

print('Model trained')


## Store sample input and output datasets into Azure

np.savetxt('features.csv', dataset_x, delimiter=',')
np.savetxt('labels.csv', dataset_y, delimiter=',')
run = Run.get_context()
ws = run.experiment.workspace
datastore = ws.get_default_datastore()
datastore.upload_files(files=['./features.csv', './labels.csv'],
                       target_path='sklearn_regression/',
                       overwrite=True)

input_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/features.csv')])
output_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/labels.csv')])

## Register model

model = Model.register(workspace=ws,
                       model_name='my-sklearn-model',                # Name of the registered model in your workspace.
                       model_path='./sklearn_regression_model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version=sklearn.__version__,  # Version of scikit-learn used to create the model.
                       sample_input_dataset=input_dataset,
                       sample_output_dataset=output_dataset,
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='Ridge regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

In [ ]:
from azureml.core import Experiment, ScriptRunConfig
experiment = Experiment(workspace=ws, name='day2-experiment-train')
config = ScriptRunConfig(source_directory='.',
                             script='train.py',
                             compute_target='cpu-cluster',
                             environment=myenv)
run = experiment.submit(config)

In [ ]:
aml_url = run.get_portal_url()
print(aml_url)